# VRP Demo with SCIP
This file is intended to build the model for the CVRP using SCIP

In [9]:
from pyscipopt import Model, Pricer, SCIP_RESULT, SCIP_STAGE
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import networkx as nx
import random
import matplotlib.pyplot as plt
import numpy as np

from cffi import FFI
ffi = FFI()
labelling_lib = ffi.dlopen("Labelling/labelling_lib.so")

funDefs = "void initGraph(unsigned num_nodes, unsigned* node_data, double* edge_data, const double capacity); void labelling(double const * dual,const bool farkas, unsigned* result);"
ffi.cdef(funDefs, override=True)

In [10]:
# Create Graph
G = nx.complete_graph(12)
for (u, v) in G.edges():
    G.edges[u,v]['weight'] = random.randint(1,10)
    
for node in G.nodes():
    G.nodes()[node]['demand'] = random.randint(1,10)

# nx.draw(G)

In [11]:
# Create Simple Graph for correctnes
G = nx.complete_graph(4)
G.remove_edge(0,3)
for (u, v) in G.edges():
    G.edges[u,v]['weight'] = 1
G.edges[1,2]['weight'] = 1

for node in G.nodes():
    G.nodes()[node]['demand'] = 1

# nx.draw(G,with_labels=True)

In [12]:
class VRP(Model):
    def __init__(self,graph):
        super().__init__()
        
        self.original_graph = graph
        self.graph = graph.copy()
        self.vars = {}
        self.cons = []

In [20]:
class VRPPricer(Pricer): 
    def pricerinit(self):
        self.data['cons'] = [self.model.getTransformedCons(con) for con in self.model.cons]
        self.data['vars'] = {path:self.model.getTransformedVar(var) for (path,var) in self.model.vars.items()}
        
        print(f" There are {len(self.model.getConss())} constraints in the model and {len(self.data['cons'])} of them are known to the pricer.")
        
        node_data = list(nx.get_node_attributes(self.model.graph,"demand").values())
        nodes_arr = ffi.cast("unsigned*", np.array(node_data).astype(np.uintc).ctypes.data)

        edges = nx.adjacency_matrix(self.model.graph,dtype=np.double).toarray()
        edges_arr = ffi.cast("double*", edges.ctypes.data)
        
        num_nodes = ffi.cast("unsigned",self.model.graph.number_of_nodes())
        
        capacity_ptr = ffi.cast("double",self.data['capacity'])
        labelling_lib.initGraph(num_nodes,nodes_arr,edges_arr, capacity_ptr)
    
    def pricerfarkas(self):
#         print("Farkas Pricing has been called.")
        dual = [self.model.getDualfarkasLinear(con) for con in self.data['cons']]
        print(f"Farkas Values are {dual}")
        return self.labelling(dual, farkas=True)

    def pricerredcost(self):
        pi = [self.model.getDualsolLinear(con) for con in self.data['cons']]
        print(f"Dual variables are {pi}")
        return self.labelling(pi)
    
    def labelling(self, dual,farkas=False):
        pointer_dual = ffi.cast("double*", np.array(dual,dtype=np.double).ctypes.data)
        
        # TODO: Possible improvement: result can be reused every time
        result = np.zeros(self.data['capacity'] + 2,dtype=np.uintc)
        result_arr = ffi.cast("unsigned*",result.ctypes.data)
        
        labelling_lib.labelling(pointer_dual, False,result_arr)
        
        if(result[0] == 1):
            print("There are no paths with negative reduced costs")
            # There are no paths with negative reduced costs
            return {'result':SCIP_RESULT.SUCCESS}
        
        print(result)
        result_indices = np.insert(np.nonzero(result),0,0)
#         print(np.insert(np.nonzero(result),0,0))
        path = tuple(result[result_indices])
        print(f"Labelling found path {path} with negative reduced cost")
        if path in self.data['vars'].keys():
            cost = self.model.getVarRedcost(self.data['vars'][path])
            print(f"Path already in variables with reduced cost {cost} with farkas {farkas}.")
        
        var = self.model.addVar(vtype="B",obj=nx.path_weight(self.model.graph,path,"weight"),pricedVar=True)
        weight = nx.path_weight(self.model.graph,path,"weight")
        print(f"Added {path} with path weight {weight}")
        counts = np.unique(path[1:-1], return_counts=True)
        for i, node in enumerate(counts[0]):
            print(f"There are {counts[1][i]} occurences of node {node}")
            self.model.addConsCoeff(self.data['cons'][node-1], var ,counts[1][i])

        self.model.addConsCoeff(self.data['cons'][-1], var, 1)
        self.data['vars'][tuple(path)] = var
        
        return {'result':SCIP_RESULT.SUCCESS}

In [23]:
model = VRP(G)
num_vehicles = 3

# Create pricer
pricer = VRPPricer()
pricer.data = {}
pricer.data["capacity"] = 3
pricer.data["num_vehicles"] = num_vehicles
model.includePricer(pricer, "pricer","does pricing")

# Create a valid set of variables and the constraints to it
for i in range(1,G.number_of_nodes()-1):
    path = (0,i,G.number_of_nodes()-1)
    cost = nx.path_weight(G,path,"weight")
    var = model.addVar(vtype="B",obj=cost)
    model.vars[path] = var
    cons = model.addCons(var == 1, name=str(node),modifiable=True)
    model.cons.append(cons)
    
# Add the convexity constraint, which limits the number of available vehicles
convexity_constraint = model.addCons(sum(model.vars.values()) <= num_vehicles, modifiable=True)
model.cons.append(convexity_constraint)

model.hideOutput()
model.optimize()
model.hideOutput(quiet=False)
model.printBestSol()
print(pricer.data['vars'])

 There are 3 constraints in the model and 3 of them are known to the pricer.
Graph data successfully copied to C.
Dual variables are [2.0, 2.0, -0.0]
[0 1 2 3 0]
Labelling found path (0, 1, 2, 3) with negative reduced cost
Added (0, 1, 2, 3) with path weight 3
There are 1 occurences of node 1
There are 1 occurences of node 2
Dual variables are [2.0, 2.0, -0.0]
[0 1 2 3 0]
Labelling found path (0, 1, 2, 3) with negative reduced cost
Path already in variables with reduced cost -1.0 with farkas False.
Added (0, 1, 2, 3) with path weight 3
There are 1 occurences of node 1
There are 1 occurences of node 2
Dual variables are [2.0, 1.0, -0.0]
[0 1 2 1 3]
Labelling found path (0, 1, 2, 1, 3) with negative reduced cost
Added (0, 1, 2, 1, 3) with path weight 4
There are 2 occurences of node 1
There are 1 occurences of node 2
Dual variables are [1.0, 2.0, -0.0]
[0 2 1 2 3]
Labelling found path (0, 2, 1, 2, 3) with negative reduced cost
Added (0, 2, 1, 2, 3) with path weight 4
There are 1 occurenc

Dual variables are [1.3333333333333333, 1.3333333333333335, -0.0]
There are no paths with negative reduced costs
Farkas Values are [0.0, -1.0, 0.0]
There are no paths with negative reduced costs
Dual variables are [1.0, 2.0, -0.0]
[0 2 1 2 3]
Labelling found path (0, 2, 1, 2, 3) with negative reduced cost
Path already in variables with reduced cost -1.0 with farkas False.
Added (0, 2, 1, 2, 3) with path weight 4
There are 1 occurences of node 1
There are 2 occurences of node 2
Dual variables are [1.3333333333333333, 1.3333333333333335, -0.0]
There are no paths with negative reduced costs
Farkas Values are [0.0, -1.0, 0.0]
There are no paths with negative reduced costs
Dual variables are [1.0, 2.0, -0.0]
[0 2 1 2 3]
Labelling found path (0, 2, 1, 2, 3) with negative reduced cost
Path already in variables with reduced cost -1.0 with farkas False.
Added (0, 2, 1, 2, 3) with path weight 4
There are 1 occurences of node 1
There are 2 occurences of node 2
Dual variables are [1.33333333333333

Dual variables are [1.3333333333333333, 1.3333333333333335, -0.0]
There are no paths with negative reduced costs
Farkas Values are [0.0, -1.0, 0.0]
There are no paths with negative reduced costs
Dual variables are [1.0, 2.0, -0.0]
[0 2 1 2 3]
Labelling found path (0, 2, 1, 2, 3) with negative reduced cost
Path already in variables with reduced cost -1.0 with farkas False.
Added (0, 2, 1, 2, 3) with path weight 4
There are 1 occurences of node 1
There are 2 occurences of node 2
Dual variables are [1.3333333333333333, 1.3333333333333335, -0.0]
There are no paths with negative reduced costs
Farkas Values are [0.0, -1.0, 0.0]
There are no paths with negative reduced costs
Dual variables are [1.0, 2.0, -0.0]
[0 2 1 2 3]
Labelling found path (0, 2, 1, 2, 3) with negative reduced cost
Path already in variables with reduced cost -1.0 with farkas False.
Added (0, 2, 1, 2, 3) with path weight 4
There are 1 occurences of node 1
There are 2 occurences of node 2
Dual variables are [1.33333333333333

[0 2 1 2 3]
Labelling found path (0, 2, 1, 2, 3) with negative reduced cost
Path already in variables with reduced cost -1.0 with farkas False.
Added (0, 2, 1, 2, 3) with path weight 4
There are 1 occurences of node 1
There are 2 occurences of node 2
Dual variables are [1.3333333333333333, 1.3333333333333335, -0.0]
There are no paths with negative reduced costs
Farkas Values are [0.0, -1.0, 0.0]
There are no paths with negative reduced costs
Dual variables are [1.0, 2.0, -0.0]
[0 2 1 2 3]
Labelling found path (0, 2, 1, 2, 3) with negative reduced cost
Path already in variables with reduced cost -1.0 with farkas False.
Added (0, 2, 1, 2, 3) with path weight 4
There are 1 occurences of node 1
There are 2 occurences of node 2
Dual variables are [1.3333333333333333, 1.3333333333333335, -0.0]
There are no paths with negative reduced costs
Farkas Values are [0.0, -1.0, 0.0]
There are no paths with negative reduced costs
Dual variables are [1.0, 2.0, -0.0]
[0 2 1 2 3]
Labelling found path (0,

There are no paths with negative reduced costs
Dual variables are [1.0, 2.0, -0.0]
[0 2 1 2 3]
Labelling found path (0, 2, 1, 2, 3) with negative reduced cost
Path already in variables with reduced cost -1.0 with farkas False.
Added (0, 2, 1, 2, 3) with path weight 4
There are 1 occurences of node 1
There are 2 occurences of node 2
Dual variables are [1.3333333333333333, 1.3333333333333335, -0.0]
There are no paths with negative reduced costs
Farkas Values are [0.0, -1.0, 0.0]
There are no paths with negative reduced costs
Dual variables are [1.0, 2.0, -0.0]
[0 2 1 2 3]
Labelling found path (0, 2, 1, 2, 3) with negative reduced cost
Path already in variables with reduced cost -1.0 with farkas False.
Added (0, 2, 1, 2, 3) with path weight 4
There are 1 occurences of node 1
There are 2 occurences of node 2
Dual variables are [1.3333333333333333, 1.3333333333333335, -0.0]
There are no paths with negative reduced costs
Farkas Values are [0.0, -1.0, 0.0]
There are no paths with negative red

...

There are 2 occurences of node 2
Dual variables are [1.3333333333333333, 1.3333333333333335, -0.0]
There are no paths with negative reduced costs
Farkas Values are [0.0, -1.0, 0.0]
There are no paths with negative reduced costs
Dual variables are [1.0, 2.0, -0.0]
[0 2 1 2 3]
Labelling found path (0, 2, 1, 2, 3) with negative reduced cost
Path already in variables with reduced cost -1.0 with farkas False.
Added (0, 2, 1, 2, 3) with path weight 4
There are 1 occurences of node 1
There are 2 occurences of node 2
Dual variables are [1.3333333333333333, 1.3333333333333335, -0.0]
There are no paths with negative reduced costs
Farkas Values are [0.0, -1.0, 0.0]
There are no paths with negative reduced costs
Dual variables are [1.0, 2.0, -0.0]
[0 2 1 2 3]
Labelling found path (0, 2, 1, 2, 3) with negative reduced cost
Path already in variables with reduced cost -1.0 with farkas False.
Added (0, 2, 1, 2, 3) with path weight 4
There are 1 occurences of node 1
There are 2 occurences of node 2
Dua

Farkas Values are [0.0, -1.0, 0.0]
There are no paths with negative reduced costs
Dual variables are [1.0, 2.0, -0.0]
[0 2 1 2 3]
Labelling found path (0, 2, 1, 2, 3) with negative reduced cost
Path already in variables with reduced cost -1.0 with farkas False.
Added (0, 2, 1, 2, 3) with path weight 4
There are 1 occurences of node 1
There are 2 occurences of node 2
Dual variables are [1.3333333333333333, 1.3333333333333335, -0.0]
There are no paths with negative reduced costs
Farkas Values are [0.0, -1.0, 0.0]
There are no paths with negative reduced costs
Dual variables are [1.0, 2.0, -0.0]
[0 2 1 2 3]
Labelling found path (0, 2, 1, 2, 3) with negative reduced cost
Path already in variables with reduced cost -1.0 with farkas False.
Added (0, 2, 1, 2, 3) with path weight 4
There are 1 occurences of node 1
There are 2 occurences of node 2
Dual variables are [1.3333333333333333, 1.3333333333333335, -0.0]
There are no paths with negative reduced costs
Farkas Values are [0.0, -1.0, 0.0]
T

There are no paths with negative reduced costs
Farkas Values are [0.0, -1.0, 0.0]
There are no paths with negative reduced costs
Dual variables are [1.0, 2.0, -0.0]
[0 2 1 2 3]
Labelling found path (0, 2, 1, 2, 3) with negative reduced cost
Path already in variables with reduced cost -1.0 with farkas False.
Added (0, 2, 1, 2, 3) with path weight 4
There are 1 occurences of node 1
There are 2 occurences of node 2
Dual variables are [1.3333333333333333, 1.3333333333333335, -0.0]
There are no paths with negative reduced costs
Farkas Values are [0.0, -1.0, 0.0]
There are no paths with negative reduced costs
Dual variables are [1.0, 2.0, -0.0]
[0 2 1 2 3]
Labelling found path (0, 2, 1, 2, 3) with negative reduced cost
Path already in variables with reduced cost -1.0 with farkas False.
Added (0, 2, 1, 2, 3) with path weight 4
There are 1 occurences of node 1
There are 2 occurences of node 2
Dual variables are [1.3333333333333333, 1.3333333333333335, -0.0]
There are no paths with negative red

Labelling found path (0, 2, 1, 2, 3) with negative reduced cost
Path already in variables with reduced cost -1.0 with farkas False.
Added (0, 2, 1, 2, 3) with path weight 4
There are 1 occurences of node 1
There are 2 occurences of node 2
Dual variables are [1.3333333333333333, 1.3333333333333335, -0.0]
There are no paths with negative reduced costs
Farkas Values are [0.0, -1.0, 0.0]
There are no paths with negative reduced costs
Dual variables are [1.0, 2.0, -0.0]
[0 2 1 2 3]
Labelling found path (0, 2, 1, 2, 3) with negative reduced cost
Path already in variables with reduced cost -1.0 with farkas False.
Added (0, 2, 1, 2, 3) with path weight 4
There are 1 occurences of node 1
There are 2 occurences of node 2
Dual variables are [1.3333333333333333, 1.3333333333333335, -0.0]
There are no paths with negative reduced costs
Farkas Values are [0.0, -1.0, 0.0]
There are no paths with negative reduced costs
Dual variables are [1.0, 2.0, -0.0]
[0 2 1 2 3]
Labelling found path (0, 2, 1, 2, 3)

x3                                                  1 	(obj:3)


## Old Labelling

In [ ]:

    def simple_labeling(self,dual,farkas=False):
        for path in nx.all_simple_paths(self.model.graph,0,list(G.nodes())[-1]):
            if tuple(path) not in self.data['vars'].keys():
                if farkas:
                    weight = 0
                else:
                    weight = nx.path_weight(self.model.graph,path,"weight")
                load = 0
                
                for node in path[1:-1]:
                    weight -= dual[node-1]
                    load += self.model.graph.nodes()[node]['demand']
                if weight < 0 and load <= self.data['capacity']:
                    var = self.model.addVar(vtype="B",obj=nx.path_weight(self.model.graph,path,"weight"),pricedVar=True)
                    for node in path[1:-1]:
                        self.model.addConsCoeff(self.data['cons'][node-1], var ,1)
                    
                    self.model.addConsCoeff(self.data['cons'][-1], var, 1)
                    self.data['vars'][tuple(path)] = var
#                     print(f"Found path {path} with negative reduced cost")
                    return {'result':SCIP_RESULT.SUCCESS}
        return {'result':SCIP_RESULT.SUCCESS}